# 10. 다대다 RNN을 이용한 텍스트 생성

## 1. 문자 단위 RNN(Char RNN)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### 1. 훈련 데이터 전처리하기

In [2]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


In [5]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [8]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)
index_to_char = dict((i, c) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(index_to_char)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [7]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [9]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [10]:
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [11]:
## np.eye 값이 1인 대각 행렬 생성
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [12]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [13]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


## 2. 모델 구현하기

In [14]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [15]:
net = Net(input_size, hidden_size, output_size)

In [16]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([1, 5, 5])


In [24]:
print(outputs)

tensor([[[ 0.1641, -0.0765,  0.1594,  0.1471, -0.1795],
         [-0.1640, -0.1142,  0.4702, -0.2702, -0.3927],
         [-0.3306, -0.4906,  0.1867, -0.2972, -0.2036],
         [-0.3007, -0.1731,  0.4266, -0.1313, -0.2599],
         [ 0.1802, -0.3486, -0.0091, -0.1549, -0.2095]]],
       grad_fn=<AddBackward0>)


In [20]:
print(outputs.view(-1, input_size).shape)

torch.Size([5, 5])


In [25]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [26]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [27]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.595436453819275 prediction:  [[0 2 2 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  !eee!
1 loss:  1.280083179473877 prediction:  [[3 2 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  lele!
2 loss:  1.0261132717132568 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
3 loss:  0.8056618571281433 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
4 loss:  0.6222957372665405 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
5 loss:  0.4772101938724518 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.35604363679885864 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.2681226134300232 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.19888095557689667 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.1425553262233734 prediction:  [[4 4 3 2 0]] 

## 02. 문자 단위 RNN(Char RNN) - 더 많은 데이터

### 2. 문자 단위 RNN(Char RNN)

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim

#### 1. 훈련 데이터 전처리하기

In [30]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [31]:
print(sentence)

if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.


In [35]:
char_set = sorted(list(set(sentence)))
char_dic = {c: i for i, c in enumerate(char_set)} 

In [36]:
print(char_dic)

{' ': 0, "'": 1, ',': 2, '.': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'w': 23, 'y': 24}


In [38]:
dic_size = len(char_dic)
print('문자 집합의 크기:{}'.format(dic_size))

문자 집합의 크기:25


In [39]:
hidden_size = dic_size
sequence_length = 10
learning_rate = 0.1

In [56]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i: i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    #print(i, x_str, '->', y_str)
    
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])    

In [57]:
print(x_data[0])
print(y_data[0])

[12, 9, 0, 24, 17, 22, 0, 23, 4, 16]
[9, 0, 24, 17, 22, 0, 23, 4, 16, 21]


In [58]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [59]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [60]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,

In [61]:
print(Y[0])

tensor([ 9,  0, 24, 17, 22,  0, 23,  4, 16, 21])


#### 2. 모델 구현하기

In [62]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [63]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

In [64]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [65]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


In [66]:
print(outputs.view(-1, dic_size).shape)

torch.Size([1700, 25])


In [67]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [68]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

bb..bb.b.p..p.p.bb.....b.b.b..bp.....bb.....bp..bp..pp..b....p....p...p.....p....p....bbbbp.b.b....pb...b.ppb.b.p.b......b...bp.....b....p..p.pb.....p.bp.kpp..bbbppbpb.p.bp..b.bb.
iyyyolyooiyeolllkalkooyke lkklooykollkkklkolyklllye lyynryeololkekoykkololkolkolookkeoywaliyollkeolkolyolkooloykolkilooyeoiyolollkkeoyeolllkolfo lyyeoyyekolykolykoykaiwooykoloyyei
 oooo  nonon n on o nooon nooon non    oo on n onooo  n n non o   o  on o ooo ooo oono  ooooon nono n  o  oo  o n non n oo ooo n o o  ooo n o  no ooo oon ono on  o ooo nonooon  o 
  tugpn'teteeeeeeeu eeeeeeeeteeeeeeetetegeegte eete eeeee ee geeeeeeegee teee eeegteeu eaeegteeeeeeeetene teeegeeuteeeeetetetteeeeeeeteeeeeeeeee  teeggee eeteene tteeeeet eeeetee 
   t  e  e   e t    t     t   tt  t  e teete    e         e  te   t  e     te t     e  e  ee  e t  te t  e e  t t   t      t  te t  e te  e    e tt  e e  te  t         t   e e t r
a he hethrrer roreoreree rerrr rererrorererrrrrrrrrere errer eorerrrrerrr rrrrereoheeorerreorereeorr

### 03. 단어 단위 RNN - 임베딩 사용

#### 1. 훈련 데이터 전처리하기

In [69]:
import torch
import torch.nn as nn
import torch.optim as optim

In [70]:
sentence = "Repeat is the best medicine for memory".split()

In [71]:
vocab = list(set(sentence))
print(vocab)

['medicine', 'the', 'Repeat', 'memory', 'best', 'for', 'is']


In [72]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [73]:
print(word2index)

{'medicine': 1, 'the': 2, 'Repeat': 3, 'memory': 4, 'best': 5, 'for': 6, 'is': 7, '<unk>': 0}


In [74]:
print(word2index['memory'])

4


In [75]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'medicine', 2: 'the', 3: 'Repeat', 4: 'memory', 5: 'best', 6: 'for', 7: 'is', 0: '<unk>'}


In [79]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [81]:
X, Y = build_data(sentence, word2index)

[3, 7, 2, 5, 1, 6, 4]


In [82]:
print(X)
print(Y)

tensor([[3, 7, 2, 5, 1, 6]])
tensor([[7, 2, 5, 1, 6, 4]])


### 2. 모델 구현하기

In [83]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [84]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [85]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [86]:
output = model(X)
print(output)

tensor([[ 0.0968, -0.2711,  0.0053,  0.0386,  0.0768, -0.2800,  0.2467,  0.1175],
        [-0.0008, -0.3563, -0.2970,  0.0030,  0.0381,  0.1386, -0.1654, -0.3065],
        [ 0.1185, -0.2048, -0.4029,  0.0622,  0.2628, -0.0447,  0.3870, -0.2292],
        [ 0.0748, -0.2192, -0.0489,  0.0833,  0.1840,  0.0554,  0.1279,  0.2357],
        [ 0.1727, -0.1640, -0.2139,  0.5631, -0.0592, -0.0676,  0.1717,  0.1887],
        [-0.1189, -0.4375, -0.2494,  0.3765,  0.2510,  0.3500, -0.1968,  0.2485]],
       grad_fn=<ViewBackward>)


In [87]:
print(output.shape)

torch.Size([6, 8])


In [88]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [89]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.1134 
Repeat for best for is Repeat Repeat

[41/201] 1.5081 
Repeat is the best medicine for memory

[81/201] 0.8274 
Repeat is the best medicine for memory

[121/201] 0.3798 
Repeat is the best medicine for memory

[161/201] 0.1993 
Repeat is the best medicine for memory

[201/201] 0.1234 
Repeat is the best medicine for memory

